In [40]:
import os, re
from pyspark.sql.types import *
import pandas as pd

In [10]:
from pyspark.sql import SparkSession
# May take a little while on a local computer
spark = SparkSession.builder.appName("music_box").getOrCreate()

# load play_log

In [11]:
play_path = '/Users/wanjiewang/PycharmProjects/music_box/music_box/processed_data/play/'
play_list = []
for fname in os.listdir(play_path):
    play_list.append(play_path+fname)

In [12]:
play_list

['/Users/wanjiewang/PycharmProjects/music_box/music_box/processed_data/play/.DS_Store',
 '/Users/wanjiewang/PycharmProjects/music_box/music_box/processed_data/play/20170401_1_play.csv',
 '/Users/wanjiewang/PycharmProjects/music_box/music_box/processed_data/play/20170401_2_play.csv',
 '/Users/wanjiewang/PycharmProjects/music_box/music_box/processed_data/play/20170401_3_play.csv',
 '/Users/wanjiewang/PycharmProjects/music_box/music_box/processed_data/play/20170402_1_play.csv',
 '/Users/wanjiewang/PycharmProjects/music_box/music_box/processed_data/play/20170402_2_play.csv',
 '/Users/wanjiewang/PycharmProjects/music_box/music_box/processed_data/play/20170402_3_play.csv',
 '/Users/wanjiewang/PycharmProjects/music_box/music_box/processed_data/play/20170403_1_play.csv',
 '/Users/wanjiewang/PycharmProjects/music_box/music_box/processed_data/play/20170403_2_play.csv',
 '/Users/wanjiewang/PycharmProjects/music_box/music_box/processed_data/play/20170404_1_play.csv',
 '/Users/wanjiewang/PycharmPro

In [13]:
# load all play log:
play_schema = StructType([ StructField("uid",IntegerType(),nullable = False), 
                          StructField("new_device", StringType()), 
                          StructField("song_id",IntegerType()), 
                          StructField("new_play_time", DoubleType()),
                          StructField("new_song_length", DoubleType()),
                          StructField("date", DateType())
                        ])

In [14]:
for i in range(len(play_list)):
    if i == 0:
        df_play = spark.read.csv(play_list[i],sep = '|', header = True, schema = play_schema, encoding = 'UTF-8')
    else:
        df_play_merge = spark.read.csv(play_list[i],sep = '|', header = True, schema = play_schema, encoding = 'UTF-8')
        df_play = df_play.unionAll(df_play_merge)

df_play.printSchema()

print ("Total Rows: " + str(df_play.count()))

root
 |-- uid: integer (nullable = true)
 |-- new_device: string (nullable = true)
 |-- song_id: integer (nullable = true)
 |-- new_play_time: double (nullable = true)
 |-- new_song_length: double (nullable = true)
 |-- date: date (nullable = true)

Total Rows: 98039208


In [15]:
df_play.show(5)

+---------+----------+-------+-------------+---------------+----------+
|      uid|new_device|song_id|new_play_time|new_song_length|      date|
+---------+----------+-------+-------------+---------------+----------+
| 38830551|        ar|1152372|        171.0|          172.0|2017-04-01|
|167923158|        ar| 985228|        251.0|          251.0|2017-04-01|
|168045107|        ar|6818758|          0.0|         1683.0|2017-04-01|
|168024853|        ar|3971731|        300.0|          300.0|2017-04-01|
|167580792|        ar|6989313|        113.0|          113.0|2017-04-01|
+---------+----------+-------+-------------+---------------+----------+
only showing top 5 rows



In [91]:
from pyspark.sql.functions import *

In [69]:
df_play = df_play.withColumn('mon',month(df_play['date']))
df_play = df_play.withColumn('dayofMon',dayofmonth(df_play['date']))
df_play = df_play.withColumn('3dayofMon',(dayofmonth(df_play['date'])-1)/3)
df_play = df_play.withColumn('5dayofMon',(dayofmonth(df_play['date'])-1)/5)
df_play = df_play.withColumn('7dayofMon',(dayofmonth(df_play['date'])-1)/7)
df_play = df_play.withColumn('14dayofMon',(dayofmonth(df_play['date'])-1)/14)

In [70]:
df_play.show(5)

+---------+----------+-------+-------------+---------------+----------+---+--------+---------+---------+---------+----------+
|      uid|new_device|song_id|new_play_time|new_song_length|      date|mon|dayofMon|3dayofMon|5dayofMon|7dayofMon|14dayofMon|
+---------+----------+-------+-------------+---------------+----------+---+--------+---------+---------+---------+----------+
| 38830551|        ar|1152372|        171.0|          172.0|2017-04-01|  4|       1|      0.0|      0.0|      0.0|       0.0|
|167923158|        ar| 985228|        251.0|          251.0|2017-04-01|  4|       1|      0.0|      0.0|      0.0|       0.0|
|168045107|        ar|6818758|          0.0|         1683.0|2017-04-01|  4|       1|      0.0|      0.0|      0.0|       0.0|
|168024853|        ar|3971731|        300.0|          300.0|2017-04-01|  4|       1|      0.0|      0.0|      0.0|       0.0|
|167580792|        ar|6989313|        113.0|          113.0|2017-04-01|  4|       1|      0.0|      0.0|      0.0|    

## freq_1d


In [135]:
df_play_Apr = df_play.filter(df_play['mon']==4)
df_play_May = df_play.filter(df_play['mon']==5)


In [136]:
gb_1d = df_play_Apr.groupBy('uid')

In [137]:
freq_1d_song = gb_1d.agg({'song_id':'count'}).alias('song')

In [138]:
freq_1d_playtime = gb_1d.agg({'new_play_time':'sum'}).alias('playtime')

In [139]:
df_final = freq_1d_song.join(freq_1d_playtime,col('song.uid') == col('playtime.uid'),'inner')

In [140]:
df_final = df_final.withColumn('freq_1d_song',df_final['count(song_id)']/30)
df_final = df_final.withColumn('freq_1d_playtime',df_final['sum(new_play_time)']/30)
df_final = df_final.select(['song.uid','freq_1d_song','freq_1d_playtime']).alias('final')

## freq_3d, freq_5d, freq_7d

In [141]:
df_3d_song = df_play_Apr.groupBy(['uid','3dayofMon']).agg({'song_id':'count'}).groupBy('uid').agg({'count(song_id)':'sum'}).alias('3dsong')
df_3d_play = df_play_Apr.groupBy(['uid','3dayofMon']).agg({'new_play_time':'sum'}).groupBy('uid').agg({'sum(new_play_time)':'sum'}).alias('3dplay')

df_temp = df_3d_song.join(df_3d_play,col('3dsong.uid') == col('3dplay.uid'),'inner')
df_temp = df_temp.withColumn('freq_3d_song',df_temp['sum(count(song_id))']/10)
df_temp = df_temp.withColumn('freq_3d_playtime',df_temp['sum(sum(new_play_time))']/10)
df_temp = df_temp.select(['3dsong.uid','freq_3d_song','freq_3d_playtime']).alias('3d')
df_final = df_final.join(df_temp,col('final.uid')==col('3d.uid'),'inner').select(['final.uid','freq_1d_song','freq_1d_playtime','freq_3d_song','freq_3d_playtime']).alias('final')


In [142]:
df_5d_song = df_play_Apr.groupBy(['uid','5dayofMon']).agg({'song_id':'count'}).groupBy('uid').agg({'count(song_id)':'sum'}).alias('5dsong')
df_5d_play = df_play_Apr.groupBy(['uid','5dayofMon']).agg({'new_play_time':'sum'}).groupBy('uid').agg({'sum(new_play_time)':'sum'}).alias('5dplay')

df_temp = df_5d_song.join(df_5d_play,col('5dsong.uid') == col('5dplay.uid'),'inner')
df_temp = df_temp.withColumn('freq_5d_song',df_temp['sum(count(song_id))']/6)
df_temp = df_temp.withColumn('freq_5d_playtime',df_temp['sum(sum(new_play_time))']/6)
df_temp = df_temp.select(['5dsong.uid','freq_5d_song','freq_5d_playtime']).alias('5d')
df_final = df_final.join(df_temp,col('final.uid')==col('5d.uid'),'inner').select(['final.uid','freq_1d_song','freq_1d_playtime','freq_3d_song','freq_3d_playtime','freq_5d_song','freq_5d_playtime']).alias('final')


In [143]:
df_7d_song = df_play_Apr.groupBy(['uid','7dayofMon']).agg({'song_id':'count'}).groupBy('uid').agg({'count(song_id)':'sum'}).alias('7dsong')
df_7d_play = df_play_Apr.groupBy(['uid','7dayofMon']).agg({'new_play_time':'sum'}).groupBy('uid').agg({'sum(new_play_time)':'sum'}).alias('7dplay')

df_temp = df_7d_song.join(df_7d_play,col('7dsong.uid') == col('7dplay.uid'),'inner')
df_temp = df_temp.withColumn('freq_7d_song',df_temp['sum(count(song_id))']/4)
df_temp = df_temp.withColumn('freq_7d_playtime',df_temp['sum(sum(new_play_time))']/4)
df_temp = df_temp.select(['7dsong.uid','freq_7d_song','freq_7d_playtime']).alias('7d')
df_final = df_final.join(df_temp,col('final.uid')==col('7d.uid'),'inner').select(['final.uid','freq_1d_song','freq_1d_playtime','freq_3d_song','freq_3d_playtime','freq_5d_song','freq_5d_playtime','freq_7d_song','freq_7d_playtime']).alias('final')


In [144]:
df_final.show(5)

+--------+-------------------+------------------+------------+----------------+------------------+------------------+------------+----------------+
|     uid|       freq_1d_song|  freq_1d_playtime|freq_3d_song|freq_3d_playtime|      freq_5d_song|  freq_5d_playtime|freq_7d_song|freq_7d_playtime|
+--------+-------------------+------------------+------------+----------------+------------------+------------------+------------+----------------+
|10906795|0.16666666666666666|40.166666666666664|         0.5|           120.5|0.8333333333333334|200.83333333333334|        1.25|          301.25|
|13277485| 0.5333333333333333|46.233333333333334|         1.6|           138.7|2.6666666666666665|231.16666666666666|         4.0|          346.75|
|13610475| 49.666666666666664|            3006.6|       149.0|          9019.8|248.33333333333334|           15033.0|       372.5|         22549.5|
|20974764|                1.9|             226.1|         5.7|           678.3|               9.5|            11

429392

In [146]:
df_final.write.save('play_log_freq')

# label

In [147]:
df_play_May = df_play.filter(df_play['mon']==5)

In [156]:
df_play_May.show(5)

+---------+----------+-------+-------------+---------------+----------+---+--------+---------+---------+---------+----------+
|      uid|new_device|song_id|new_play_time|new_song_length|      date|mon|dayofMon|3dayofMon|5dayofMon|7dayofMon|14dayofMon|
+---------+----------+-------+-------------+---------------+----------+---+--------+---------+---------+---------+----------+
|167942200|        ar| 442218|        260.0|          260.0|2017-05-01|  5|       1|      0.0|      0.0|      0.0|       0.0|
|167682905|        ar| 440614|          4.0|          253.0|2017-05-01|  5|       1|      0.0|      0.0|      0.0|       0.0|
|168009486|        ar| 157606|        288.0|          288.0|2017-05-01|  5|       1|      0.0|      0.0|      0.0|       0.0|
|168022196|        ar| 835317|          3.0|          260.0|2017-05-01|  5|       1|      0.0|      0.0|      0.0|       0.0|
|167879980|        ar|  90861|        300.0|          300.0|2017-05-01|  5|       1|      0.0|      0.0|      0.0|    

In [160]:
df_used = df_play_May.select('uid').alias('active')

In [158]:
df_used.show(5)

+---------+
|      uid|
+---------+
|167942200|
|167682905|
|168009486|
|168022196|
|167879980|
+---------+
only showing top 5 rows



In [179]:
test = df_final.join(df_used,col('final.uid') == col('active.uid'),'left')

In [163]:
test.show(5)

+--------+-------------------+------------------+------------+----------------+------------------+------------------+------------+----------------+--------+
|     uid|       freq_1d_song|  freq_1d_playtime|freq_3d_song|freq_3d_playtime|      freq_5d_song|  freq_5d_playtime|freq_7d_song|freq_7d_playtime|     uid|
+--------+-------------------+------------------+------------+----------------+------------------+------------------+------------+----------------+--------+
|10906795|0.16666666666666666|40.166666666666664|         0.5|           120.5|0.8333333333333334|200.83333333333334|        1.25|          301.25|    null|
|13277485| 0.5333333333333333|46.233333333333334|         1.6|           138.7|2.6666666666666665|231.16666666666666|         4.0|          346.75|    null|
|13610475| 49.666666666666664|            3006.6|       149.0|          9019.8|248.33333333333334|           15033.0|       372.5|         22549.5|13610475|
|13610475| 49.666666666666664|            3006.6|       14

In [180]:
from functools import reduce  # For Python 3.x
from pyspark.sql import DataFrame

def unionAll(*dfs):
    return reduce(DataFrame.unionAll, dfs) 



In [181]:
df1 = test.where(col('active.uid').isNotNull())
df2 = test.where(col('active.uid').isNull())

In [182]:
df1 = df1.withColumn('label',lit(1))
df2 = df2.withColumn('label',lit(0))
res = unionAll(*[df1, df2])

In [184]:
res = res.select(['final.uid','freq_1d_song','freq_1d_playtime','freq_3d_song','freq_3d_playtime','freq_5d_song','freq_5d_playtime','freq_7d_song','freq_7d_playtime','label'])

In [185]:
res.show(5)

+--------+------------------+----------------+------------+----------------+------------------+----------------+------------+----------------+-----+
|     uid|      freq_1d_song|freq_1d_playtime|freq_3d_song|freq_3d_playtime|      freq_5d_song|freq_5d_playtime|freq_7d_song|freq_7d_playtime|label|
+--------+------------------+----------------+------------+----------------+------------------+----------------+------------+----------------+-----+
|13610475|49.666666666666664|          3006.6|       149.0|          9019.8|248.33333333333334|         15033.0|       372.5|         22549.5|    1|
|13610475|49.666666666666664|          3006.6|       149.0|          9019.8|248.33333333333334|         15033.0|       372.5|         22549.5|    1|
|13610475|49.666666666666664|          3006.6|       149.0|          9019.8|248.33333333333334|         15033.0|       372.5|         22549.5|    1|
|13610475|49.666666666666664|          3006.6|       149.0|          9019.8|248.33333333333334|         15

In [186]:
res.printSchema()

root
 |-- uid: integer (nullable = true)
 |-- freq_1d_song: double (nullable = true)
 |-- freq_1d_playtime: double (nullable = true)
 |-- freq_3d_song: double (nullable = true)
 |-- freq_3d_playtime: double (nullable = true)
 |-- freq_5d_song: double (nullable = true)
 |-- freq_5d_playtime: double (nullable = true)
 |-- freq_7d_song: double (nullable = true)
 |-- freq_7d_playtime: double (nullable = true)
 |-- label: integer (nullable = false)



# transform to a dataframe can be used by ml lib

In [187]:
from pyspark.ml.feature import VectorAssembler


In [202]:
res.columns

['uid',
 'freq_1d_song',
 'freq_1d_playtime',
 'freq_3d_song',
 'freq_3d_playtime',
 'freq_5d_song',
 'freq_5d_playtime',
 'freq_7d_song',
 'freq_7d_playtime',
 'label']

In [203]:
assembler = VectorAssembler(
  inputCols=['freq_1d_song',
 'freq_1d_playtime',
 'freq_3d_song',
 'freq_3d_playtime',
 'freq_5d_song',
 'freq_5d_playtime',
 'freq_7d_song',
 'freq_7d_playtime'],
              outputCol="features")

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:53312)
Traceback (most recent call last):
  File "/Users/wanjiewang/spark-2.3.0-bin-hadoop2.7/python/lib/py4j-0.10.6-src.zip/py4j/java_gateway.py", line 852, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/wanjiewang/spark-2.3.0-bin-hadoop2.7/python/lib/py4j-0.10.6-src.zip/py4j/java_gateway.py", line 990, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 61] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:53312)

In [191]:
output = assembler.transform(res)


In [199]:
output.show(5)

ERROR:py4j.java_gateway:An error occurred while trying to connect to the Java server (127.0.0.1:53312)
Traceback (most recent call last):
  File "/Users/wanjiewang/spark-2.3.0-bin-hadoop2.7/python/lib/py4j-0.10.6-src.zip/py4j/java_gateway.py", line 852, in _get_connection
    connection = self.deque.pop()
IndexError: pop from an empty deque

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/Users/wanjiewang/spark-2.3.0-bin-hadoop2.7/python/lib/py4j-0.10.6-src.zip/py4j/java_gateway.py", line 990, in start
    self.socket.connect((self.address, self.port))
ConnectionRefusedError: [Errno 61] Connection refused


Py4JNetworkError: An error occurred while trying to connect to the Java server (127.0.0.1:53312)

In [192]:
final_data = output.select("features",'label')

In [193]:
train_data,test_data = final_data.randomSplit([0.7,0.3])

In [197]:
train_data.show(5)

Py4JJavaError: An error occurred while calling o5116.showString.
: org.apache.spark.sql.catalyst.errors.package$TreeNodeException: execute, tree:
Exchange hashpartitioning(uid#2237, 200)
+- *(125) HashAggregate(keys=[uid#2237], functions=[partial_count(song_id#2239)], output=[uid#2237, count#22433L])
   +- Union
      :- *(1) Project [uid#2237, song_id#2239]
      :  +- *(1) Filter ((month(date#2242) = 4) && isnotnull(uid#2237))
      :     +- *(1) FileScan csv [uid#2237,song_id#2239,date#2242] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/Users/wanjiewang/PycharmProjects/music_box/music_box/processed_data/play/..., PartitionFilters: [], PushedFilters: [IsNotNull(uid)], ReadSchema: struct<uid:int,song_id:int,date:date>
      :- *(2) Project [uid#2249, song_id#2251]
      :  +- *(2) Filter ((month(date#2254) = 4) && isnotnull(uid#2249))
      :     +- *(2) FileScan csv [uid#2249,song_id#2251,date#2254] Batched: false, Format: CSV, Location: InMemoryFileIndex[file:/Users

In [194]:
from pyspark.ml.classification import DecisionTreeClassifier,GBTClassifier,RandomForestClassifier

In [195]:
dtc = DecisionTreeClassifier(labelCol='label',featuresCol='features')
# rfc = RandomForestClassifier(labelCol='label',featuresCol='features')
# gbt = GBTClassifier(labelCol='label',featuresCol='features')

In [196]:
# Train the models (its three models, so it might take some time)
dtc_model = dtc.fit(train_data)
# rfc_model = rfc.fit(train_data)
# gbt_model = gbt.fit(train_data)

Py4JJavaError: An error occurred while calling o5118.fit.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 220 in stage 235.0 failed 1 times, most recent failure: Lost task 220.0 in stage 235.0 (TID 44107, localhost, executor driver): java.lang.OutOfMemoryError: Java heap space
	at org.apache.spark.util.collection.unsafe.sort.UnsafeSorterSpillWriter.<init>(UnsafeSorterSpillWriter.java:52)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter$SpillableIterator.spill(UnsafeExternalSorter.java:536)
	at org.apache.spark.util.collection.unsafe.sort.UnsafeExternalSorter.spill(UnsafeExternalSorter.java:200)
	at org.apache.spark.memory.TaskMemoryManager.acquireExecutionMemory(TaskMemoryManager.java:180)
	at org.apache.spark.memory.TaskMemoryManager.allocatePage(TaskMemoryManager.java:283)
	at org.apache.spark.memory.TaskMemoryManager.allocatePage(TaskMemoryManager.java:310)
	at org.apache.spark.memory.MemoryConsumer.allocatePage(MemoryConsumer.java:117)

In [ ]:
dtc_predictions = dtc_model.transform(test_data)

# evaluation

In [ ]:
from pyspark.ml.evaluation import BinaryClassificationEvaluatoriclassClassificationEvaluator

In [ ]:
acc_evaluator = MulticlassClassificationEvaluator(labelCol="label", predictionCol="prediction", metricName="accuracy")

In [ ]:
dtc_acc = acc_evaluator.evaluate(dtc_predictions)